# Info
원본 코드 : https://github.com/jwkirchenbauer/lm-watermarking

참고 설명 : https://colab.research.google.com/drive/1V8eMbM6LZNOKADDZZFoXQ2ukdGQ4dkbl?usp=drive_link

## Watermark AI-Generated Texts

- 대규모 언어 모델(LLM)의 워터마킹 및 탐지 알고리즘

- AI로 생성된 텍스트의 출력을 추적하고 진위를 확인하기 위해 워터마크를 추가하는 기술.

### A Watermark for Large Language Models

https://github.com/jwkirchenbauer/lm-watermarking

In [1]:
#트랜스포머 모델 이용 = accelerate 설치
!pip install -q accelerate
#코드 클론
!git clone -q https://github.com/jwkirchenbauer/lm-watermarking.git lm_watermarking
#확인
%cd lm_watermarking

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.5 MB/s eta 0:00:00
/content/lm_watermarking


## LLM 로드
https://huggingface.co/Qwen/Qwen2-1.5B-Instruct

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

# 알리바바에서 만든 Qwen2모델 사용
# AutoModelForCausalLM.from_pretrained( ) :Hugging Face의 Transformers 라이브러리에서 제공하는 함수
# 모델 불러오는 함수
# device_map : CPU 또는 GPU에 배치할 지를 결정
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
# AutoTokenizer.from_pretrained() : Hugging Face의 Transformers 라이브러리에서 제공하는 함수
# 문장을 토큰으로 분리하고 각 토큰에 대해 인덱스를 할당합니다.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 샘플 텍스트 생성

In [3]:
prompt = "Large Language Model에 대해 간략하게 설명해주세요."
messages = [
    {"role": "system", "content": "당신은 친절한 챗봇입니다."},
    {"role": "user", "content": prompt}
]
# tokenizer.apply_chat_template() : 대화 스크립트를 처리하여 특정 형식으로 변환하는 메서드입니다.
# tokenize=False는 토크나이징을 하지 말라는 옵션을 의미
# add_generation_prompt=True는 생성 프롬프트를 추가하라는 옵션입니다.
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# tokenizer([text], return_tensors="pt"): 변환된 텍스트를 모델이 이해할 수 있는 형식으로 토큰화하여 텐서로 변환하는 메서드입니다.
#"pt"는 PyTorch 텐서를 의미합니다.
#.to(device)는 데이터를 지정된 장치 (예: GPU)로 이동시키는 메서드입니다.
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# model.generate(): 모델에 입력 데이터를 제공하여 새로운 텍스트를 생성하는 메서드입니다.
# model_inputs.input_ids는 모델의 입력 토큰 ID를 의미
# max_new_tokens=128은 생성할 최대 토큰 수를 제한하는 옵션입니다.
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128
)

# 생성된 출력에서 입력 토큰 ID를 제외한 부분을 추출하는 리스트 컴프리헨션입니다.
# 이를 통해 모델이 생성한 새로운 토큰들을 가져옵니다.
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# tokenizer.batch_decode(): 생성된 토큰 ID를 텍스트로 디코딩하는 메서드입니다.
# skip_special_tokens=True는 특수 토큰을 건너뛰도록 설정한 옵션입니다.
# [0]은 배치에서 첫 번째 결과만 가져오도록 합니다.
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

대규모 언어 모델(Large Language Model, LLM)은 자연어 처리(Natural Language Processing, NLP) 분야에서 중요한 기술 중 하나입니다. 이들은 다양한 언어를 이해하고, 그들의 의미와 문맥을 파악하여 정확한 응답을 제공할 수 있습니다.

LLM의 주요 역할은 인공지능(AI) 시스템이 사용하는 대화형 AI나 지식 그래프 모델(Knowledge Graph Models) 등과 같은 뛰어난 능력을 갖추는 것입니다. 이러한 모델은 많은 언어


## 워터마크 안한 것 테스트

In [4]:
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
# WatermarkDetector()사용
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting.
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0, # 워터마크 검증(확인) 임계값. 조정하면서 테스트 가능
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

# z_score와 prediction(결과값)확인
print(score_dict)

{'num_tokens_scored': 125, 'num_green_tokens': 27, 'green_fraction': 0.216, 'z_score': -0.8778762251403478, 'p_value': 0.809994553675125, 'z_score_at_T': tensor([-0.5774, -0.8165,  0.3333,  0.0000, -0.2582, -0.4714, -0.6547, -0.8165,
        -0.1925, -0.3651, -0.5222, -0.6667, -0.8006, -0.9258, -1.0435, -0.5774,
        -0.1400, -0.2722,  0.1325,  0.5164,  0.3780,  0.2462,  0.1204,  0.0000,
        -0.1155, -0.2265, -0.3333,  0.0000, -0.1072, -0.2108, -0.3111, -0.4082,
        -0.5025, -0.5941, -0.2928,  0.0000, -0.0949, -0.1873, -0.2774, -0.3651,
        -0.4508, -0.5345, -0.6163, -0.6963, -0.7746, -0.8513, -0.9264, -1.0000,
        -1.0722, -0.8165, -0.8893, -0.9608, -1.0310, -0.7857, -0.8563, -0.9258,
        -0.9941, -1.0613, -1.1275, -1.1926, -0.9610, -1.0265, -1.0911, -0.8660,
        -0.6445, -0.4264, -0.4937, -0.2801, -0.0695, -0.1380, -0.2056, -0.2722,
        -0.3379, -0.4027, -0.4667, -0.5298, -0.5922, -0.6537, -0.7145, -0.7746,
        -0.8340, -0.6376, -0.6971, -0.7559, -0

## 워터마크

In [5]:
from transformers import LogitsProcessorList
#  WatermarkLogitsProcessor(): 단어를 재배열하여 동작
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="selfhash") # equivalent to `ff-anchored_minhash_prf-4-True-15485863`

In [6]:
prompt = "Large Language Model에 대해 간략하게 설명해주세요."
messages = [
    {"role": "system", "content": "당신은 친절한 챗봇입니다."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128,
    # 기존에서 추가된 코
    logits_processor=LogitsProcessorList([watermark_processor])
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

"Large Language Model"이라는 표현은 인공지능에서 사용되는 다양한 기술과 프로그램을 떠올리게 만드는 일반적인 용어입니다. 대체적으로, "Llm" 또는 "LLM"라는 단어는 모델이 너무 크고 복잡하여 그 자체가 한 가지 모듈이나 시스템보다 더 많은 데이터와 정보를 처리할 수 있는 능력을 의미합니다.

특정 기업이나 연구소에서는 이러한 "large language model"을 통해 고급 Natural Language Processing (NLP) 기술을 개발하거나 공개합니다


In [8]:
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0,
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

print(score_dict)

{'num_tokens_scored': 125, 'num_green_tokens': 62, 'green_fraction': 0.496, 'z_score': 6.351692687780163, 'p_value': 1.0647922005016939e-10, 'z_score_at_T': tensor([-0.5774,  0.8165,  1.6667,  1.1547,  0.7746,  0.4714,  0.2182,  0.0000,
        -0.1925,  0.3651,  0.8704,  0.6667,  1.1209,  0.9258,  0.7454,  0.5774,
         0.9802,  0.8165,  1.1921,  1.5492,  1.8898,  2.2156,  2.0466,  1.8856,
         2.1939,  2.0381,  2.3333,  2.1822,  2.4659,  2.7406,  2.5924,  2.4495,
         2.3116,  2.5744,  2.8301,  3.0792,  2.9424,  2.8098,  2.6811,  2.5560,
         2.7952,  2.6726,  2.9055,  3.1334,  3.0123,  3.2348,  3.4528,  3.3333,
         3.5466,  3.7559,  3.6380,  3.5228,  3.7273,  3.9284,  3.8146,  4.0119,
         4.2060,  4.3970,  4.2844,  4.1740,  4.3614,  4.5461,  4.7281,  4.9075,
         4.7980,  4.6904,  4.8669,  4.7610,  4.9348,  5.1065,  5.2760,  5.4433,
         5.6086,  5.5035,  5.6667,  5.5630,  5.7242,  5.8835,  6.0410,  6.1968,
         6.0943,  5.9932,  5.8936,  5.7955,

## 문장의 일부만 넣어도 가능한가

In [9]:
response = """대형 언어 모델 (Large Language Model)은 정보를 바탕으로 새로운 문장이나 추론을 생성하는 능력을 가지고 있습니다.

특정 주제의 다양한 정보를 이해하려면, 언어 모델은 수많은 데이터와 사전 배운 정보에 입각하여 학습해야 합니다. 대형 언어 모델은 일반적으로 이들을 더 효과적으로 통합하며"""

watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0,
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

print(score_dict)

{'num_tokens_scored': 80, 'num_green_tokens': 21, 'green_fraction': 0.2625, 'z_score': 0.2581988897471611, 'p_value': 0.3981267073688196, 'z_score_at_T': tensor([-0.5774,  0.8165,  0.3333,  0.0000,  0.7746,  0.4714,  0.2182,  0.0000,
        -0.1925, -0.3651, -0.5222, -0.6667, -0.1601, -0.3086, -0.4472, -0.5774,
        -0.7001, -0.8165, -0.9272, -1.0328, -1.1339, -1.2309, -1.3242, -1.4142,
        -1.0392, -0.6794, -0.7778, -0.4364, -0.5361, -0.6325, -0.7259, -0.8165,
        -0.9045, -0.9901, -0.6831, -0.7698, -0.8542, -0.9366, -0.6472, -0.7303,
        -0.8115, -0.8909, -0.9685, -1.0445, -0.7746, -0.7746, -0.8513, -0.5895,
        -0.3333, -0.4124, -0.4899, -0.2425, -0.3203, -0.3965, -0.4714, -0.5449,
        -0.3086, -0.3824, -0.4549, -0.2255, -0.2981, -0.3696, -0.1466, -0.2182,
        -0.2887, -0.3581, -0.4264, -0.4937, -0.2801, -0.3475, -0.4140, -0.4140,
        -0.4140, -0.4140, -0.2056,  0.0000,  0.2027,  0.4027,  0.3333,  0.5298,
         0.4606,  0.3922,  0.3248,  0.2582]), 

In [10]:
response = """대형 언어 모델 (Large Language Model)은 정보를 바탕으로 새로운 문장이나 추론을 생성하는 능력을 가지고 있습니다.

특정 주제의 다양한 정보를 이해하려면, 언어 모델은 수많은 데이터와 사전 배운 정보에 입각하여 학습해야 합니다. 대형 언어 모델은 일반적으로 이들을"""

watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25, # should match original setting
    seeding_scheme="selfhash", # should match original setting
    device=model.device, # must match the original rng device type
    tokenizer=tokenizer,
    z_threshold=4.0,
    normalizers=[],
    ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(response)

print(score_dict)

{'num_tokens_scored': 74, 'num_green_tokens': 20, 'green_fraction': 0.2702702702702703, 'z_score': 0.4026936331284146, 'p_value': 0.34358680915482404, 'z_score_at_T': tensor([-0.5774,  0.8165,  0.3333,  0.0000,  0.7746,  0.4714,  0.2182,  0.0000,
        -0.1925, -0.3651, -0.5222, -0.6667, -0.1601, -0.3086, -0.4472, -0.5774,
        -0.7001, -0.8165, -0.9272, -1.0328, -1.1339, -1.2309, -1.3242, -1.4142,
        -1.0392, -0.6794, -0.7778, -0.4364, -0.5361, -0.6325, -0.7259, -0.8165,
        -0.9045, -0.9901, -0.6831, -0.7698, -0.8542, -0.9366, -0.6472, -0.7303,
        -0.8115, -0.8909, -0.9685, -1.0445, -0.7746, -0.7746, -0.8513, -0.5895,
        -0.3333, -0.4124, -0.4899, -0.2425, -0.3203, -0.3965, -0.4714, -0.5449,
        -0.3086, -0.3824, -0.4549, -0.2255, -0.2981, -0.3696, -0.1466, -0.2182,
        -0.2887, -0.3581, -0.4264, -0.4937, -0.2801, -0.3475, -0.4140, -0.4140,
        -0.4140, -0.4140, -0.2056,  0.0000,  0.2027,  0.4027]), 'prediction': False}


# 결과
문자열을 많이 지울 수록 워터마크를 추축하기가 힘들다.

# 워터마크를 피하는 법

1. 챗지피티에서 생성된 텍스트를 다르게 재구성함으로써, 콘텐츠에 적용된 워터마크를 뒤엎을 수 있습니다.
2.생성된 텍스트를 다시 작성하는 보조 AI 모델을 사용하는 것입니다. 이러한 방법을 사용하면 워터마크 시퀀스가 깨지기 때문에 AI 생성 콘텐츠로 감지하기 어려워집니다.

# ChatGPT가 작성한 글인지 어떻게 알 수 있나요?


ChatGPT로 작성된 글인지 판단하기는 어렵지만, 일부 패턴을 인식하는 데는 AI 탐지기 도구가 필요하지 않습니다.

지금 읽고 있는 콘텐츠가 ChatGPT에서 제작되었다는 것을 알 수 있는 8가지는 다음과 같습니다.


##1. 패턴과 불일치 찾기

앞서 언급했듯이 AI가 생성한 텍스트에는 인식 가능한 패턴이 많이 있습니다.

이는 텍스트의 구문이나 구조의 패턴뿐만 아니라 텍스트 자체가 얼마나 예측 가능한지에 대한 패턴이어야 합니다.

AI는 틀에서 벗어난 사고를 할 수 없으며, 오히려 읽고 이해하기 쉬운 안전하고 일관된 결과물을 제공하고자 합니다.

지나치게 단순하고 예측 가능한 언어는 AI가 만든 텍스트의 특징입니다.

##2. 인적 오류의 징후 확인
잘 모르시겠지만, 사람이 작성한 텍스트에는 대부분 오류가 포함되어 있습니다.

이는 지극히 정상적인 현상이며 수 세기 동안 이어져 온 관행입니다.

편집자가 있는 사람이 글을 쓰면 잘못 배치된 구두점이나 문법 또는 시제 문제 같은 것을 놓칠 수 있습니다.

하지만 AI 텍스트에 비해 사람의 글은 더 세련되고 자연스럽습니다.

ChatGPT와 같은 AI 도구는 텍스트를 작성하는 것이 아니라 학습된 모든 데이터에서 텍스트를 생성합니다.

여기에는 우리에게는 어색하게 들리지만 AI 도구 자체에는 식별할 수 없는 문법이나 단어 선택의 오류가 포함될 수 있습니다.

##3. 설명 언어가 부족한지 살펴보세요.
AI가 생성한 콘텐츠의 난해성 점수가 바로 이 점수에 해당합니다.

대부분의 LLM은 사용자가 혼동하지 않도록 설계 및 프로그래밍되어 있습니다. 이러한 이유로 이러한 도구는 더 간단하고 이해하기 쉬운 언어를 사용합니다.

또한 단어 선택에 있어서도 다양성이 부족하며, 같은 단어나 문구가 같은 텍스트 내에서 여러 번 나타나는 경우가 많기 때문에 이를 알 수 있습니다.

##4. 컨텍스트에 주의하기
문맥에 주의를 기울여야 하는 이유는 무엇일까요? AI 도구는 맥락을 제대로 제공하지 못하는 경우가 많기 때문입니다.

AI는 기본적으로 이전 데이터 세트에서 출력과 응답을 생성한다는 점을 기억하세요.

각 대화의 새로운 맥락을 ChatGPT가 지속적으로 이해하는 것은 쉽지 않습니다.

ChatGPT는 점점 더 똑똑해지고 있으며, 이전 대화의 맥락을 이해하고 학습할 수 있습니다.

하지만 대화의 텍스트 출력에 있어서는 ChatGPT 및 기타 AI 도구는 여전히 인간 작성자가 쉽게 포함할 수 있는 문맥을 제공하는 데는 상당히 미흡합니다.

##5. 과도한 전환어 사용
전환 단어는 문장과 아이디어를 하나로 묶는 데 탁월합니다.

이러한 구조는 인간 작가에게는 자연스러운 것이지만, AI 도구의 경우 과도하게 전환어로 보완하는 것 같습니다.

이러한 LLM은 유사한 문장 구조와 길이를 제공하도록 학습됩니다.

이를 방지하기 위해 문장이나 개념을 부자연스러운 방식으로 묶으려는 과도한 알고리즘도 있는 것 같습니다.

텍스트를 읽었을 때 이상하게 들린다면 해당 텍스트의 제작에 인공지능이 관여했을 가능성이 높습니다.

##6. 옳은 것 같지만 말이 안 되는 문장
겉보기에는 맞지만 문장을 이해하려고 할 때 약간 어긋나는 문장을 본 적이 있다면 AI가 작성한 것일 가능성이 높습니다.

이러한 도구는 문장 구조에 관해서는 잘 훈련되어 있지만 문장의 주제에 관해서는 종종 놓칠 수 있습니다.

ChatGPT 및 기타 AI 도구는 어디선가 학습한 데이터를 되풀이할 뿐이므로 어색한 구절과 문장이 나올 수 있다는 점을 항상 기억하세요.

##7. 독창성 부족
대부분의 인공지능 텍스트를 읽어보면 독창성이 전혀 느껴지지 않습니다.

인간 작가의 뉘앙스와 영혼이 빠져 있습니다. Google의 EEAT 가이드라인으로 인해 AI가 생성한 콘텐츠가 높은 순위를 차지하기 어려운 데에는 그만한 이유가 있습니다.

Google은 콘텐츠에서 경험, 전문성, 권위성, 신뢰성을 보여주는 사례를 찾고 있습니다.

이 모든 것은 인공지능이 제공하기 어려운 것들, 특히 독창적이고 생각을 자극하는 방식으로 제공하기 어려운 것들입니다.

심지어 다른 출처의 텍스트를 완전히 표절한 AI 텍스트의 예도 있었습니다.

AI 도구는 매우 유용하지만 인간에 비해 독창성이 부족합니다.

##8. 사실 오류
이미 AI 환각에 대해 간략하게 살펴봤지만, 안타깝게도 여전히 AI 글쓰기 도구로 비즈니스를 수행하는 데 드는 비용입니다.

ChatGPT가 실제로 부정확한 출력을 제공하는 경우, 그 진위 여부와 정확성을 확인하는 것은 사용자의 책임입니다.

ChatGPT 자체는 눈 하나 깜짝하지 않고 사실 오류를 제공할 수 있습니다.

텍스트에서 특정 내용이 노골적으로 틀린 것을 발견했다면, AI 도구나 끔찍하게 잘못된 정보를 가진 사람이 작성했을 가능성이 높습니다.

# 생성AI감지 툴

- SEO.ai

무료 AI 탐지 도구도 제공합니다. 이 도구는 최신 GPT-3.5 및 GPT-4 버전의 ChatGPT와 Google Bard와 같은 기타 LLM을 감지할 수 있습니다.

- Copyleaks

- GPTZero : ChatGPT 또는 다른 LLM에 의해 작성되었는지 감지하는 데 사용할 수 있는 AI 도구입니다.